In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy
%matplotlib inline

In [2]:
data_path = '/Users/Super/Documents/GitHub/plane_schedule/全国经纬度.csv'

In [3]:
pd.read_csv(data_path, delim_whitespace=True)

,省份,地市,区县,经度,纬度
0,北京市,北京市,北京市,116.40,39.90
1,北京市,北京市,天安门,116.38,39.90
2,北京市,北京市,东城区,116.42,39.93
3,北京市,北京市,西城区,116.37,39.92
4,北京市,北京市,崇文区,116.43,39.88
5,北京市,北京市,宣武区,116.35,39.87
6,北京市,北京市,朝阳区,116.43,39.92
7,北京市,北京市,丰台区,116.28,39.85
8,北京市,北京市,石景山区,116.22,39.90
9,北京市,北京市,海淀区,116.30,39.95


In [4]:
data_original = pd.read_csv(data_path, delim_whitespace=True)

In [ ]:
# 计算城市之间的距离
def distance_ij()